## Imported Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.stats import ttest_ind
pd.set_option("display.max_rows", 200)

ModuleNotFoundError: No module named 'numpy'

## Read CSV File

In [4]:
df = pd.read_csv("FIES PUF 2012 Vol.1.CSV")
df

NameError: name 'pd' is not defined

In [59]:
# Get only people who have jobs
selected_df = df[["W_REGN","W_OID","W_SHSN","W_HCN","NONAGRI_SAL","EMPLOYED_PAY","OCCUP","JOB","SEX"]]

W_REGN                    int64
W_OID                     int64
W_SHSN                    int64
W_HCN                     int64
URB                       int64
RSTR                      int64
PSU                       int64
BWEIGHT                 float64
RFACT                   float64
FSIZE                   float64
AGRI_SAL                  int64
NONAGRI_SAL               int64
WAGES                     int64
NETSHARE                  int64
CASH_ABROAD               int64
CASH_DOMESTIC             int64
RENTALS_REC               int64
INTEREST                  int64
PENSION                   int64
DIVIDENDS                 int64
OTHER_SOURCE              int64
NET_RECEIPT               int64
REGFT                     int64
NET_CFG                   int64
NET_LPR                   int64
NET_FISH                  int64
NET_FOR                   int64
NET_RET                   int64
NET_MFG                   int64
NET_COM                   int64
NET_TRANS                 int64
NET_MIN 

## Data Cleaning
* W_REGN
* NONAGRI_SAL
* EMPLOYED_PAY
* OCCUP
* JOB

In [91]:
selected_df = selected_df.drop_duplicates()

,OCCUP,JOB
0,,2
1,1314,1
2,,2
3,6111,1
4,5220,1
5,6212,1
6,,2
7,6111,1
8,1314,1
9,6114,1


In [60]:
selected_df = selected_df.dropna(subset=["OCCUP"])

(40171, 4)

In [ ]:
selected_df = selected_df[selected_df["JOB"] != 2]

In [ ]:
selected_df = selected_df[selected_df["NONAGRI_SAL"] > 0]

In [ ]:
selected_df.loc[:, "EMPLOYED_PAY"] = pd.to_numeric(selected_df["EMPLOYED_PAY"], errors='coerce')
selected_df.loc[:, "EMPLOYED_PAY"] = selected_df["EMPLOYED_PAY"].fillna(0).astype(int)
employed_df = selected_df[["EMPLOYED_PAY"]]

In [ ]:
selected_df = selected_df[selected_df["EMPLOYED_PAY"] == 1]

In [84]:
clean_df = selected_df
clean_df

W_REGN
13    377,550.15
41    263,845.54
3     257,516.48
14    250,204.47
1     205,236.51
7     202,649.68
6     201,246.62
2     192,994.08
11    189,222.12
10    181,611.94
Name: TOINC, dtype: object

## EDA Questions

* How does the average salary of the top job type in each region compare to the region's overall average salary?
* Which region has the highest overall average salary for non-agricultural jobs?
* What are the most common occupations in the dataset, and how many people are employed in each occupation across different regions?
* Is there a notable difference in the salary distribution by gender for sole earners in non-agricultural roles?

Q1: How does the average salary of the top job type in each region compare to the region's overall average salary?

In [2]:
region_avg_salary = clean_df.groupby("W_REGN")["NONAGRI_SAL"].mean().reset_index()
region_avg_salary.columns = ['W_REGN', 'AVG_SAL']

NameError: name 'df' is not defined

In [ ]:
job_type_avg_salary = clean_df.groupby(["W_REGN", "OCCUP"])["NONAGRI_SAL"].mean().reset_index()

top_job_by_region = job_type_avg_salary.loc[job_type_avg_salary.groupby('W_REGN')['NONAGRI_SAL'].idxmax()]
top_job_by_region.columns = ['W_REGN', 'Top_Job_Type', 'Top_Job_Avg_Salary']
top_job_by_region.head(20)

In [ ]:
comparison_df = pd.merge(region_avg_salary, top_job_by_region, on="W_REGN")
comparison_df

In [ ]:
comparison_df['difference'] = comparison_df['Top_Job_Avg_Salary'] - comparison_df['AVG_SAL']
comparison_df

Q2: Which Region has the highest overall average salary for non-agricultural jobs?

In [ ]:
grouped_regions = clean_df.groupby(["W_REGN"])
grouped_regions = grouped_regions.agg({"NONAGRI_SAL" : "mean"})
grouped_regions

In [ ]:
highest_mean_salary = grouped_regions["NONAGRI_SAL"].max()
highest_region = grouped_regions["NONAGRI_SAL"].idxmax()
print(f"Region {highest_region} has the highest mean non-agricultural salary: ", "{:.2f}".format(highest_mean_salary))

Q3: What are the top 10 common occupations in the dataset, and how many people are employed in each occupation across different regions?
-- mode for each occupation
-- sort
-- get top 10

In [ ]:
print("These are the top 10 most common occupation numbers")
occupations_df = clean_df['OCCUP']
occupations_df.value_counts().head(10)

Q4: Is there a notable difference in the salary distribution by gender for sole earners in non-agricultural roles?

In [ ]:
print("H0: There is no significant difference in the salary distribution of male and female sole earners in non-agricultural roles")
print("HA: There is a significant difference in the salary distribution of male and female sole earners in non-agricultural roles")
Sex1 = clean_df[clean_df["SEX"] == 1]["NONAGRI_SAL"]
Sex2 = clean_df[clean_df["SEX"] == 2]["NONAGRI_SAL"]
t_stat, p_value = stats.ttest_ind(Sex1, Sex2)

print(f"T-statistic: {t_stat}, P-value: {p_value}")
print("Answer: Do not reject the null hypothesis, there is no significant difference in the salary distribution of male and female sole earners in non-agricultural roles")